# Import Packages

In [3]:
import pandas as pd
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import re  

# Import Data

In [4]:
Category_Repos = pd.read_csv ('Category_KW_Respository.csv')
user_rating_pred = pd.read_csv('Rating_Prediction.csv')
book_info = pd.read_csv('Additional dataset_for_tag&KW_recommendation.csv')

book_info = book_info.drop( ['Unnamed: 0','count'],axis=1).sort_values(by = 'Avg Rating', ascending = False)
Category_Repos = Category_Repos.drop( ['Unnamed: 0'],axis = 1)

FileNotFoundError: [Errno 2] No such file or directory: 'Rating_Prediction.csv'

In [ ]:
# Lets Check Every Imported Dataset 

In [ ]:
# Display full (non - truncated dataframe)
#pd.set_option('display.max_colwidth', -1)

In [ ]:
book_info.head()

In [ ]:
Category_Repos.head(2)

In [ ]:
user_rating_pred.head()

# Non Personalized Recommendations for New Users 

In [ ]:
def non_personalized_recommend(cat = None, auth = None, n = 5):
    
    #if only number of top n books is searched 
    if (cat == None) and (auth == None):
        Books = book_info
    if cat != None:
    # Filter for Category
        if cat in list(Category_Repos.Category):
            Books = book_info[book_info.tag_name.str.match(pat = cat)]
        else:
            all_selected_rows = pd.DataFrame()
            Books = pd.DataFrame()
            for item in Category_Repos['Possible Search KW']:
                if cat in item:
                     Boolean_Mask = Category_Repos['Possible Search KW']==item
                     select_row  = Category_Repos[ Boolean_Mask]# Category_Repos[Category_Repos.Category.str.match(pat = str(Category))]
                     all_selected_rows = pd.concat([all_selected_rows, select_row])
            Categories = list(all_selected_rows.Category)
            for Category in Categories:                                   
                book_row = book_info[book_info.tag_name.str.match(pat = str( Category))]
                Books = pd.concat([Books,book_row])
            
            
    # Filter for authors
    if auth!= None:
        if cat!= None:
            author_names = list(set(Books.authors))
            authors = []
            for author_name in author_names:
                if auth in author_name:
                    authors.append(author_name)
            Books_filtered_auth = pd.DataFrame() 
            for author in list(authors):                                   
                book_row_au = Books[Books.authors.str.match(pat = str(author))]
                Books_filtered_auth = pd.concat([Books_filtered_auth,book_row_au])
            Books =  Books_filtered_auth
        else:
            Books = book_info[book_info.authors.str.match(pat = auth)]
    result = Books.sort_values(by = 'Avg Rating',ascending = False) [:n]
    return result.drop('book_id',axis =1)

## Demonstration

In [ ]:
# Top 5 books to show any user 
non_personalized_recommend(n = 5)

In [ ]:
# If a new user search by category and also is a bit lazy at putting inputs
non_personalized_recommend( cat = 'sci')

In [ ]:
# If a new user search by both category and author
non_personalized_recommend( cat = 'ya', auth ='Rowling')

# Personalized Recommendation for Existing Users

In [ ]:
def recommend( user = None, cat = None, auth = None, n = 5):
    if user != None:
        Books_for_user = user_rating_pred[user_rating_pred.user_id == user] [['user_id','book_id','Predicted Rating']]
        require_columns = book_info[['book_id','title','tag_name','authors','People Often Tagged as']]
        book_repository = Books_for_user.merge(require_columns, on = 'book_id') 
       
    #if only number of top n books is searched 
        if cat == None and auth == None:
            Books = book_repository
        if cat != None:
    # Filter for Category
            if cat in list(Category_Repos.Category):
                Books = book_repository[book_repository.tag_name.str.match(pat = cat)]
            else:
                all_selected_rows = pd.DataFrame()
                Books = pd.DataFrame()
                for item in Category_Repos['Possible Search KW']:
                    if cat in item:
                         Boolean_Mask = Category_Repos['Possible Search KW']==item
                         select_row  = Category = Category_Repos[ Boolean_Mask]# Category_Repos[Category_Repos.Category.str.match(pat = str(Category))]
                         all_selected_rows = pd.concat([all_selected_rows, select_row])
                Categories = list(all_selected_rows.Category)
                for Category in Categories:                                   
                    book_row = book_repository[book_repository.tag_name.str.match(pat = str( Category))]
                    Books = pd.concat([Books,book_row])
            
    # Filter for authors
        if auth!= None:
            if cat!= None:
                author_names = list(set(Books.authors))
                authors = []
                for author_name in author_names:
                    if auth in author_name:
                        authors.append(author_name)
                Books_filtered_auth = pd.DataFrame() 
                for author in list(authors):                                   
                    book_row_au = Books[Books.authors.str.match(pat = str(author))]
                    Books_filtered_auth = pd.concat([Books_filtered_auth,book_row_au])
                Books =  Books_filtered_auth
            else:
                Books = book_repository[book_repository.authors.str.match(pat = auth)]
        result = Books.sort_values(by = 'Predicted Rating',ascending = False) [:n]
    else:
        result = non_personalized_recommend(cat,auth,n)
        
    return result

## Demonstration

In [ ]:
# Lets do a search for user 9503

# Top 5 books to show user 9503 
recommend(user =9503, n = 30)

In [ ]:
# if user 9503 likes to read Fiction
recommend(user =9503, cat = 'Fiction')

In [ ]:
# if user 9503 likes to read books from Mark Twain
recommend(user =9503, auth = 'Mark Twain')

## Smart Filtering of Keywords: Example

In [ ]:
# If a new user only remember that the author's name of the book  his friend recommended starts with 'Row'
non_personalized_recommend( cat = 'ya', auth ='Row')